# Twitter Sentiment Analysis

Scripping Twitter menggunakan library **SNScrape** [dokumentasi](https://github.com/JustAnotherArchivist/snscrape)

## 1. Initiation
Pada tahap inisiasi, akan dilakukan install library SNScrape, import library-library yang dibutuhkan, *mount* script ini ke drive, dan konfigurasi pandas

Install library SNScrape dapat dilakukan dengan script berikut:

In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 KB 1.0 MB/s eta 0:00:00


Library yang digunakan adalah sntwitter dari snscrape. pandas dan numpy untuk mempermudah proses wrangling. dan library json, ast, re untuk mempermudah parsing data atau extraksi data

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import json
import ast
import re

Mount drive dapat dilakukan menggunakan script berikut untuk dapat mempermudah upload dan download file yang diinginkan

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Konfigurasi pandas untuk mempercantik dan bisa menampilkan kolom dan row dengan jumlah lebih banyak

In [ ]:
# Pandas config
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## 2. Define Table
Dalam proses get twitter terdiri dari get data dengan kode saham "$kode" dan get data menggunakan 

In [ ]:
def get_tweets_union(kode, nama, until):
  attributes_container = []
  # Using TwitterSearchScraper to scrape data and append tweets to list
  query = '{} until:{} lang:in'.format('$'+kode, until)
  enum = enumerate(sntwitter.TwitterSearchScraper(query).get_items())
  for i,tweet in enum:
    if i >= 50000:
      print('break')
      break
    attributes_container.append(json.loads(tweet.json()))
      
  # Creating a dataframe to load the list
  tweets_dfkd = pd.DataFrame(attributes_container)

  attributes_container = []
  # Using TwitterSearchScraper to scrape data and append tweets to list
  query = '{} until:{} lang:in'.format(nama, until)
  enum = enumerate(sntwitter.TwitterSearchScraper(query).get_items())
  for i,tweet in enum:
    if i >= 50000:
      print('break')
      break
    attributes_container.append(json.loads(tweet.json()))
      
  # Creating a dataframe to load the list
  tweets_dfnm = pd.DataFrame(attributes_container)
  return pd.concat([tweets_dfkd, tweets_dfnm])

In [ ]:
FILENAME = '/content/drive/MyDrive/Freelance/twitter_sentiment_analysis/aktivitas_pencatatan_ipo.csv'
DESTFILE = '/content/drive/MyDrive/Freelance/twitter_sentiment_analysis/data_gabungan/'
df_fileipo = pd.read_csv(FILENAME)
df_fileipo['ipo_date'] = (df_fileipo['tgl_ipo'].astype('datetime64[ns]') + pd.DateOffset(1)).dt.strftime('%Y-%m-%d')
df_fileipo[df_fileipo['Kode'] == 'BUKA'].head()

,No,Kode,Nama,Sektor,tanggal_awal_poin_1,Tanggal_Penawaran_Awal_Poin_6,Tanggal_Pencatatan_poin_13,Harga_poin_6,Harga_Penawaran,Tanggal_penawaran_Umum_poin_11,refference,Delisting_Date,Number_of_Shares,Papan_Pencatatan,tahun,Tanggal_Penawaran_Awal_Poin_6_1,Tanggal_Penawaran_Awal_Poin_6_2,tanngal_awal,tgl_ipo,ipo_date
26,2021-08-06T00:00:00,BUKA,PT Bukalapak.com Tbk,Technology,NaN,9 Jul 2021 -19 Jul 2021,06 Ags 2021,Rp 750-Rp 850,Rp 850,27 Jul 2021 - 30 Jul 2021,https://investasi.kontan.co.id/,NaN,103.062.019.354,Ekonomi Baru,2021,7/9/2021,7/19/2021,5/4/2021,8/6/2021,2021-08-07


In [ ]:
# Test get GOTO
kode = 'GOTO'
nama = 'GoTo Gojek Tokopedia'
until = '2022-04-12'
df_goto = get_tweets_union(kode, nama, until)
len(df_goto)
# df_goto['Date Created'] = df_goto['Date Created'].dt.tz_localize(None)
# .to_excel(DESTFILE+kode+".xlsx")

3773

In [ ]:
# df_goto.to_excel(DESTFILE+kode+".xlsx")

In [ ]:
list_emiten = df_fileipo[['Kode', 'Nama', 'ipo_date']].T.to_dict()
is_first = True
for emiten in list_emiten.values():
  kode = emiten['Kode']
  nama = emiten['Nama'][3:-4]
  until = emiten['ipo_date']
  if kode == 'BUKA':
    nama = 'PT Bukalapak'
  elif kode == 'LUCY':
    nama = 'PT Lima Dua Lima Tiga'

  df_tweets = get_tweets_union(kode, nama, until)
  df_tweets['kode_saham'] = kode
  df_tweets['nama_saham'] = nama
  df_tweets['unitl_date'] = until

  if is_first:
    df_temp = df_tweets
    is_first = False
  else:
    df_temp = pd.concat([df_temp, df_tweets])
  
  print(kode, nama, until, len(df_tweets), len(df_temp))
  df_temp.to_excel(DESTFILE+"gab_plus_offside.xlsx")

DRMA Dharma Polimetal 2021-12-21 1159 1159
BSML Bintang Samudera Mandiri Lines 2021-12-17 21 1180
NASI Wahana Inti Makmur 2021-12-14 89 1269
IPPE Indo Pureco Pratama 2021-12-10 16 1285
OBMD OBM Drilchem 2021-12-09 26 1311
AVIA Avia Avian 2021-12-09 364 1675
RMKE RMK Energy 2021-12-08 167 1842
CMRY Cisarua Mountain Dairy 2021-12-07 1009 2851
WGSH Wira Global Solusi 2021-12-07 21 2872
TAYS Jaya Swarasa Agung 2021-12-07 90 2962
WMPP Widodo Makmur Perkasa 2021-12-07 416 3378
DEPO Caturkarda Depo Bangunan 2021-11-26 1239 4617
BINO Perma Plasindo 2021-11-26 17 4634
MTEL Dayamitra Telekomunikasi 2021-11-23 2832 7466
BOBA Formosa Ingredient Factory 2021-11-02 176 7642
KUAS Ace Oldfields 2021-10-26 64 7706
IDEA Idea Indonesia Akademi 2021-09-10 124 7830
RSGK Kedoya Adyaraya 2021-09-09 64 7894
RUNS Global Sukses Solusi 2021-09-09 164 8058
SBMA Surya Biru Murni Acetylene 2021-09-09 53 8111
CMNT Cemindo Gemilang 2021-09-09 2771 10882
GTSI GTS Internasional 2021-09-09 83 10965
MCOL Prima Andalan Ma

In [ ]:
df_tweets = pd.read_excel(DESTFILE+"gab_plus_offside.xlsx")
print(len(df_tweets))

59712

In [ ]:
user_keys = ['username',
            'id',
            'displayname',
            'rawDescription',
            'renderedDescription',
            'descriptionLinks',
            'verified',
            'created',
            'followersCount',
            'friendsCount',
            'statusesCount',
            'favouritesCount',
            'listedCount',
            'mediaCount',
            'location',
            'protected',
            'profileImageUrl',
            'profileBannerUrl',
            'label',
            'url']

for key in user_keys:
  df_tweets['user_'+key] = df_tweets['user'].apply(lambda x: ast.literal_eval(x)[key])

In [ ]:
quotedTweets = ['date',
                'rawContent',
                'id',
                'lang']
for key in quotedTweets:
  df_tweets['quotedTweets_'+key] = df_tweets['quotedTweet'].apply(lambda x: ast.literal_eval(x)[key] if str(x) != 'nan' else 'nan')
df_tweets['quotedTweets_username'] = df_tweets['quotedTweet'].apply(lambda x: ast.literal_eval(x)['user']['username'] if str(x) != 'nan' else 'nan')

df_tweets.drop(columns=['user', 'quotedTweet']).head()

,Unnamed: 0,_type,url,date,rawContent,renderedContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,links,media,retweetedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe,kode_saham,nama_saham,unitl_date,user_username,user_id,user_displayname,user_rawDescription,user_renderedDescription,user_descriptionLinks,user_verified,user_created,user_followersCount,user_friendsCount,user_statusesCount,user_favouritesCount,user_listedCount,user_mediaCount,user_location,user_protected,user_profileImageUrl,user_profileBannerUrl,user_label,user_url,quotedTweets_date,quotedTweets_rawContent,quotedTweets_id,quotedTweets_lang,quotedTweets_username
0,0,snscrape.modules.twitter.Tweet,https://twitter.com/patocheclub/status/1468373...,2021-12-08T00:15:45+00:00,1/5 - MultiDays Runners?\n$AGIL $AVCT $CADL $C...,1/5 - MultiDays Runners?\n$AGIL $AVCT $CADL $C...,1.468374e+18,0,0,0,0,1.468374e+18,in,"<a href=""https://help.twitter.com/en/using-twi...",https://help.twitter.com/en/using-twitter/how-...,Tick_Screener,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['AGIL', 'AVCT', 'CADL', 'CBAT', 'DBGI', 'DRMA...",NaN,NaN,NaN,DRMA,Dharma Polimetal,2021-12-21,patocheclub,81269258,proximus.algo,bla bla bla,bla bla bla,None,False,2009-10-10T03:16:45+00:00,82,81,2348,894,1,128,"Bruxelles, Belgique",False,https://pbs.twimg.com/profile_images/260457833...,https://pbs.twimg.com/profile_banners/81269258...,None,https://twitter.com/patocheclub,nan,nan,nan,nan,nan
1,1,snscrape.modules.twitter.Tweet,https://twitter.com/LargeVoidBot/status/146823...,2021-12-07T14:59:30+00:00,"09:59 Pops: $PAYOW +6.8%, $DRMA +6.5%, $OSS +6...","09:59 Pops: $PAYOW +6.8%, $DRMA +6.5%, $OSS +6...",1.468234e+18,0,0,0,0,1.468234e+18,in,"<a href=""http://largevoid.com/"" rel=""nofollow""...",http://largevoid.com/,LV PriceAlertBot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['PAYOW', 'DRMA', 'OSS', 'PED', 'HARP', 'NTP']",NaN,NaN,NaN,DRMA,Dharma Polimetal,2021-12-21,LargeVoidBot,412611622,LargeVoid Bot,Bot by @largevoid,Bot by @largevoid,None,False,2011-11-14T23:10:09+00:00,4469,0,379470,17,180,1,In the wires,False,https://pbs.twimg.com/profile_images/163934755...,https://pbs.twimg.com/profile_banners/41261162...,None,https://twitter.com/LargeVoidBot,nan,nan,nan,nan,nan
2,2,snscrape.modules.twitter.Tweet,https://twitter.com/Juiceboy76/status/14654404...,2021-11-29T22:00:01+00:00,$DRMA,$DRMA,1.465440e+18,0,0,1,0,1.465440e+18,in,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['DRMA'],NaN,NaN,NaN,DRMA,Dharma Polimetal,2021-12-21,Juiceboy76,1401223254430343178,Nacho Libre Trades,"“I love to trade and the Lord, Its the best“ ~...","“I love to trade and the Lord, Its the best“ ~...",None,False,2021-06-05T17:04:01+00:00,346,605,2894,4391,1,627,,False,https://pbs.twimg.com/profile_images/145814664...,https://pbs.twimg.com/profile_banners/14012232...,None,https://twitter.com/Juiceboy76,2021-11-29T20:52:51+00:00,$DRMA grinding towards some kind of sick rip h...,1465423532662439940,en,stockballa
3,3,snscrape.modules.twitter.Tweet,https://twitter.com/WarrantsTech/status/146399...,2021-11-25T22:30:00+00:00,TOP Warrant Gainers 🐂 - Nov 25 2021: \n\n$SNGX...,TOP Warrant Gainers 🐂 - Nov 25 2021: \n\n$SNGX...,1.463998e+18,0,0,0,0,1.463998e+18,in,"<a href=""https://help.twitter.com/en/using-twi...",https://help.twitter.com/en/using-twitter/how-...,Warrants Tech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['SNGX', 'BENE', 'BFRI', 'ADEX', 'RELI', 'NVTS...",NaN,NaN,NaN,DRMA,Dharma Polimetal,2021-12-21,WarrantsTech,1333058057086115842,warrants.tech,Accurate up-to-date SPAC warrant platform pric...,Accurate up-to-date SPAC warrant platform pric...,None,False,2020-11-29T14:39:50+00:00,1988,80,1691,364,61,0,New York City,False,https://pbs.twimg.com/profile_images/133305845...,None,None,https://twitter.com/WarrantsTech,nan,nan,nan,nan,nan
4,4

In [ ]:
df_tweets.to_excel(DESTFILE+"gab_plus_offside_edited.xlsx")

In [ ]:
df_fileipo = pd.read_csv(FILENAME)
df_tweets = pd.read_excel(DESTFILE+"gab_plus_offside.xlsx")

In [ ]:
df_join = df_tweets.merge(df_fileipo, left_on='kode_saham', right_on='Kode')
df_join['rawContent'] = df_join['rawContent'].apply(lambda x: re.sub('[\r\n]', ' ', str(x)))
df_join['renderedContent'] = df_join['renderedContent'].apply(lambda x: re.sub('[\r\n]', ' ', str(x)))
df_join = df_join.drop(columns=['user', 'quotedTweet'])
df_join.head()

,Unnamed: 0,_type,url,date,rawContent,renderedContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,links,media,retweetedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe,kode_saham,nama_saham,unitl_date,No,Kode,Nama,Sektor,tanggal_awal_poin_1,Tanggal_Penawaran_Awal_Poin_6,Tanggal_Pencatatan_poin_13,Harga_poin_6,Harga_Penawaran,Tanggal_penawaran_Umum_poin_11,refference,Delisting_Date,Number_of_Shares,Papan_Pencatatan,tahun,Tanggal_Penawaran_Awal_Poin_6_1,Tanggal_Penawaran_Awal_Poin_6_2,tanngal_awal,tgl_ipo
0,0,snscrape.modules.twitter.Tweet,https://twitter.com/patocheclub/status/1468373...,2021-12-08T00:15:45+00:00,1/5 - MultiDays Runners? $AGIL $AVCT $CADL $CB...,1/5 - MultiDays Runners? $AGIL $AVCT $CADL $CB...,1.468374e+18,0,0,0,0,1.468374e+18,in,"<a href=""https://help.twitter.com/en/using-twi...",https://help.twitter.com/en/using-twitter/how-...,Tick_Screener,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['AGIL', 'AVCT', 'CADL', 'CBAT', 'DBGI', 'DRMA...",NaN,NaN,NaN,DRMA,Dharma Polimetal,2021-12-21,2021-12-20T00:00:00,DRMA,PT Dharma Polimetal Tbk,Consumer Cyclicals,NaN,19 Nov 2021 - 30 Nov 2021,20 Des 2021,Rp 500 - Rp 620,Rp 500,13 Dec 2021 - 16 Dec 2021,https://www.e-ipo.co.id/,NaN,4.705.882.300,Utama,2021,11/19/2021,11/30/2021,9/15/2021,12/20/2021
1,1,snscrape.modules.twitter.Tweet,https://twitter.com/LargeVoidBot/status/146823...,2021-12-07T14:59:30+00:00,"09:59 Pops: $PAYOW +6.8%, $DRMA +6.5%, $OSS +6...","09:59 Pops: $PAYOW +6.8%, $DRMA +6.5%, $OSS +6...",1.468234e+18,0,0,0,0,1.468234e+18,in,"<a href=""http://largevoid.com/"" rel=""nofollow""...",http://largevoid.com/,LV PriceAlertBot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['PAYOW', 'DRMA', 'OSS', 'PED', 'HARP', 'NTP']",NaN,NaN,NaN,DRMA,Dharma Polimetal,2021-12-21,2021-12-20T00:00:00,DRMA,PT Dharma Polimetal Tbk,Consumer Cyclicals,NaN,19 Nov 2021 - 30 Nov 2021,20 Des 2021,Rp 500 - Rp 620,Rp 500,13 Dec 2021 - 16 Dec 2021,https://www.e-ipo.co.id/,NaN,4.705.882.300,Utama,2021,11/19/2021,11/30/2021,9/15/2021,12/20/2021
2,2,snscrape.modules.twitter.Tweet,https://twitter.com/Juiceboy76/status/14654404...,2021-11-29T22:00:01+00:00,$DRMA,$DRMA,1.465440e+18,0,0,1,0,1.465440e+18,in,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['DRMA'],NaN,NaN,NaN,DRMA,Dharma Polimetal,2021-12-21,2021-12-20T00:00:00,DRMA,PT Dharma Polimetal Tbk,Consumer Cyclicals,NaN,19 Nov 2021 - 30 Nov 2021,20 Des 2021,Rp 500 - Rp 620,Rp 500,13 Dec 2021 - 16 Dec 2021,https://www.e-ipo.co.id/,NaN,4.705.882.300,Utama,2021,11/19/2021,11/30/2021,9/15/2021,12/20/2021
3,3,snscrape.modules.twitter.Tweet,https://twitter.com/WarrantsTech/status/146399...,2021-11-25T22:30:00+00:00,TOP Warrant Gainers 🐂 - Nov 25 2021: $SNGX: ...,TOP Warrant Gainers 🐂 - Nov 25 2021: $SNGX: ...,1.463998e+18,0,0,0,0,1.463998e+18,in,"<a href=""https://help.twitter.com/en/using-twi...",https://help.twitter.com/en/using-twitter/how-...,Warrants Tech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['SNGX', 'BENE', 'BFRI', 'ADEX', 'RELI', 'NVTS...",NaN,NaN,NaN,DRMA,Dharma Polimetal,2021-12-21,2021-12-20T00:00:00,DRMA,PT Dharma Polimetal Tbk,Consumer Cyclicals,NaN,19 Nov 2021 - 30 Nov 2021,20 Des 2021,Rp 500 - Rp 620,Rp 500,13 Dec 2021 - 16 Dec 2021,https://www.e-ipo.co.id/,NaN,4.705.882.300,Utama,2021,11/19/2021,11/30/2021,9/15/2021,12/20/2021
4,4,snscrape.modules.twitter.Tweet,https://twitter.com/WarrantsTech/status/146363...,2021-11-24T22:30:00+00:00,TOP Warrant Gainers 🐂 - Nov 24 2021: $SNGX: ...,TOP Warrant Gainers 🐂 - Nov 24 2021: $SNGX: ...,1.463636e+18,0,0,1,0,1.463636e+18,in,"<a href=""https://help.twitter.com/en/using-twi...",https://help.twitter.com/en/using-twitter/how-...,Warrants Tech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['SNGX', 'ADV', 'BFRI', 'BENE', 'ADEX', 'RELI'...",NaN,NaN,NaN,DRMA,Dharma Polimetal,2021-12-21,2021-12-20T00:00:00,DRMA,PT Dharma

In [ ]:
df_join.to_excel(DESTFILE+"gab_plus_offside_edited_joined.xlsx")

In [ ]:
df_join[df_join['inReplyToTweetId'].astype('str') != 'nan'].reset_index()['url'][0]

'https://twitter.com/PWESTNYC/status/1463161471932813313'

In [ ]:

df_fileipo.head()

,No,Nama,Sektor,tanggal_awal_poin_1,Tanggal_Penawaran_Awal_Poin_6,Tanggal_Pencatatan_poin_13,Harga_poin_6,Harga_Penawaran,Tanggal_penawaran_Umum_poin_11,refference,Delisting_Date,Number_of_Shares,Papan_Pencatatan,tahun,Tanggal_Penawaran_Awal_Poin_6_1,Tanggal_Penawaran_Awal_Poin_6_2,tanngal_awal,tgl_ipo
Kode,,,,,,,,,,,,,,,,,,
DRMA,2021-12-20T00:00:00,PT Dharma Polimetal Tbk,Consumer Cyclicals,NaN,19 Nov 2021 - 30 Nov 2021,20 Des 2021,Rp 500 - Rp 620,Rp 500,13 Dec 2021 - 16 Dec 2021,https://www.e-ipo.co.id/,NaN,4.705.882.300,Utama,2021,11/19/2021,11/30/2021,9/15/2021,12/20/2021
BSML,2021-12-16T00:00:00,PT Bintang Samudera Mandiri Lines Tbk,Transportation & Logistic,NaN,16 Nov 2021 - 25 Nov 2021,16 Des 2021,Rp 100 - Rp 150,Rp 117,09 Dec 2021 - 14 Dec 2021,https://www.e-ipo.co.id/,NaN,1.850.225.000,Pengembangan,2021,11/16/2021,11/25/2021,9/10/2021,12/16/2021
NASI,2021-12-13T00:00:00,PT Wahana Inti Makmur Tbk,Consumer Non-Cyclicals,NaN,17 Nov 2021 - 23 Nov 2021,13 Des 2021,Rp 140 - Rp 160,Rp 155,07 Dec 2021 - 09 Dec 2021,https://www.e-ipo.co.id/,NaN,807.400.000,Pengembangan,2021,11/17/2021,11/23/2021,9/8/2021,12/13/2021
IPPE,2021-12-09T00:00:00,PT Indo Pureco Pratama Tbk,Consumer Non-Cyclicals,NaN,18 Nov 2021 - 25 Nov 2021,09 Des 2021,Rp 100 - Rp 110,Rp 100,02 Dec 2021 - 07 Dec 2021,https://www.e-ipo.co.id/,NaN,4.600.000.000,Pengembangan,2021,11/18/2021,11/25/2021,9/10/2021,12/9/2021
OBMD,2021-12-08T00:00:00,PT OBM Drilchem Tbk,Basic Materials,NaN,18 Nov 2021 - 22 Nov 2021,08 Des 2021,Rp 150 - Rp 180,Rp 180,02 Dec 2021 - 06 Dec 2021,https://www.e-ipo.co.id/,NaN,732.000.000,Pengembangan,2021,11/18/2021,11/22/2021,9/7/2021,12/8/2021
